In [22]:
import spotipy
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import re
import math
from tqdm import tqdm

load_dotenv('../secrets/.env', override=True)

scope = "playlist-modify-private"

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

my_id = spotify.current_user()['id']

%run useful_functions.ipynb
%run spotify_scraping.ipynb
%run spotify_scraping_playlists.ipynb

In [23]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()
search = get_spotify_search()
playlists = get_spotify_playlists()
playlist_tracks = get_spotify_playlist_tracks()

master_df = (
broadcast_playlists
.merge(broadcast_details, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.merge(search,left_on=['Artist Name','Track Name'], right_on=['Original Search Artist','Original Search Track'], how = "left", suffixes=("", " Search"))
.merge(tracks, on='Track ID', how='left', suffixes=("", " Tracks"))
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

del(shows,
    show_broadcast_details,
    broadcast_details,
    broadcast_playlists,
    search)

#set(master_df['Show Name'])

In [24]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Indie Show with Jack Saunders")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Indie Show with Jack Saunders"
description = "Songs from the Indie Show with Jack Saunders in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,0HjRzOti4wvuKwIUsjZrHM,2KZedcpHb83f2AXlBDprAV,0
0,20Dw7Jar5hUbwX5FwHdQoG,2KZedcpHb83f2AXlBDprAV,1
0,7H5WVBQdSnJ1zmu8JoD6pg,2KZedcpHb83f2AXlBDprAV,2
0,19CIsQ1HElu4XHdLCMlSqu,2KZedcpHb83f2AXlBDprAV,3
0,2VcS3oKcOPkubN9LVzZ96l,2KZedcpHb83f2AXlBDprAV,4
...,...,...,...
0,2mdaMDbjFxmxQPO90PxT5w,2KZedcpHb83f2AXlBDprAV,710
0,2JjDl1YbzaBKlhSrUXZqNM,2KZedcpHb83f2AXlBDprAV,711
0,0AIlxnJ1kFpu6wp89rjjUv,2KZedcpHb83f2AXlBDprAV,712
0,7BZWTu4Pv77TxGmmsQaSXN,2KZedcpHb83f2AXlBDprAV,713


In [38]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Sounds with Clara Amfo")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Sounds with Clara Amfo"
description = "Songs from Future Sounds with Clara Amfo in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,531KGXtBroSrOX9LVmiIgc,3VTYIgusziIIdXiWB5hB2f,0
0,40SBS57su9xLiE1WqkXOVr,3VTYIgusziIIdXiWB5hB2f,1
0,3ITvhrRB5TlEks19k9qy2k,3VTYIgusziIIdXiWB5hB2f,2
0,6RZo8T7GY9EoWZXC5i2587,3VTYIgusziIIdXiWB5hB2f,3
0,7MhomISA7KRK6U7KzeV06v,3VTYIgusziIIdXiWB5hB2f,4
...,...,...,...
0,74RWuKK8uKMbxj4KnJpEgG,3VTYIgusziIIdXiWB5hB2f,427
0,6UoIrO15w3w62Jj8kGCnR7,3VTYIgusziIIdXiWB5hB2f,428
0,5WnvDdcyrFjbIqRxiqlet1,3VTYIgusziIIdXiWB5hB2f,429
0,6JkuJREwd6rd7t1YN069LK,3VTYIgusziIIdXiWB5hB2f,430


In [39]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Artists with Jack Saunders")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Artists with Jack Saunders"
description = "Songs from Future Artists with Jack Saunders in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,3GJbXOYweOltxSiOsCNkMd,3Wo1IZcPFAfrQJzBpW8QbG,0
0,2rf9i0Enr8cw1JRME8Rsvq,3Wo1IZcPFAfrQJzBpW8QbG,1
0,3C0kO7Bc51DEcJMep1deQg,3Wo1IZcPFAfrQJzBpW8QbG,2
0,4iz5NGjt9CccLwTvp3sNT1,3Wo1IZcPFAfrQJzBpW8QbG,3
0,4qMcXVzphY85iKSxYYrtoq,3Wo1IZcPFAfrQJzBpW8QbG,4
...,...,...,...
0,4IqwX48XrtkoqvZw4hxEfa,3Wo1IZcPFAfrQJzBpW8QbG,744
0,2vT3lchY4bU1BwM69CtDyj,3Wo1IZcPFAfrQJzBpW8QbG,745
0,5BUvQtKdYrWJtdYaSoyFfG,3Wo1IZcPFAfrQJzBpW8QbG,746
0,5cRhrmpBwkZjJ7eFMJKbVM,3Wo1IZcPFAfrQJzBpW8QbG,747


In [40]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Alternative")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Alternative"
description = "Songs from Future Alternative Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,03DbkHAGXrmwFyy7bZpr3q,0U2xXmHS8QvPrX2WqGT1UH,0
0,2fmyHaJBaPrT6cf1igqLgJ,0U2xXmHS8QvPrX2WqGT1UH,1
0,5dpOtj3sd3nnXEudsgMozs,0U2xXmHS8QvPrX2WqGT1UH,2
0,01UU5ejjra9E5wh6GGwmEq,0U2xXmHS8QvPrX2WqGT1UH,3
0,39wDX7D1cIlrAnWPW1UoEj,0U2xXmHS8QvPrX2WqGT1UH,4
...,...,...,...
0,5QV6DCj4kvteYBDD9Vt1jY,0U2xXmHS8QvPrX2WqGT1UH,492
0,6L4L6bQPHojlmwBZQLQZjc,0U2xXmHS8QvPrX2WqGT1UH,493
0,5P0iRet3L91fC6SipKKsPl,0U2xXmHS8QvPrX2WqGT1UH,494
0,4ZG0EaIiFpFOaJKmT1ZjNO,0U2xXmHS8QvPrX2WqGT1UH,495


In [41]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Steve Lamacq")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Steve Lamacq"
description = "Songs from Steve Lamacq's Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,6DqfGofHKoLSAVgEZt1eF6,6Bu0jMJgdN0Emarjx98IL7,0
0,5bF00VrMY3FwnQDgoP4Gnk,6Bu0jMJgdN0Emarjx98IL7,1
0,2pugP777iY75myaoxMlsLf,6Bu0jMJgdN0Emarjx98IL7,2
0,5qKrvS3xGWwZrDCx84cnrv,6Bu0jMJgdN0Emarjx98IL7,3
0,4DR1rowKc2WeuCQS7wyQBI,6Bu0jMJgdN0Emarjx98IL7,4
...,...,...,...
0,0mF5QmsOJq5w3FqGMjycII,6Bu0jMJgdN0Emarjx98IL7,1070
0,1ZtE8ysoPZVrMrQ24fVO4f,6Bu0jMJgdN0Emarjx98IL7,1071
0,2AmkpCt3y9NBsB5UgP4S6I,6Bu0jMJgdN0Emarjx98IL7,1072
0,5tmkX2fJ7KUAnMz3W1ibEr,6Bu0jMJgdN0Emarjx98IL7,1073


In [42]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == 'New Music Fix')].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = 'New Music Fix'
description = "Songs from New Music Fix in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,77gFd5vPEcRyYNcaaIAm3k,0Cwr3XcLdGZ5ZqDtI7lL11,0
0,3leHGaQTWV36WVrLPPF3HP,0Cwr3XcLdGZ5ZqDtI7lL11,1
0,6czOM49YUiah9nGSICy7tS,0Cwr3XcLdGZ5ZqDtI7lL11,2
0,6wKNAHfGD6C4rvjJD0Lhij,0Cwr3XcLdGZ5ZqDtI7lL11,3
0,4HmCef5mBSW6mJaVtHIMq0,0Cwr3XcLdGZ5ZqDtI7lL11,4
...,...,...,...
0,1pxLVdC8sjLDA2DF3uOEuW,0Cwr3XcLdGZ5ZqDtI7lL11,514
0,0RTuA2AWVjFw0MURes4kse,0Cwr3XcLdGZ5ZqDtI7lL11,515
0,1MCrtBVXxy2p48uWwnBwEz,0Cwr3XcLdGZ5ZqDtI7lL11,516
0,7xfwMBuatcfbzTSWRfo49f,0Cwr3XcLdGZ5ZqDtI7lL11,517


In [43]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Chillest Show")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Chillest Show"
description = "Songs from the Chillest Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,6XM6FI6rPJBnhoF6heNHeN,1p5mKMwcQVCeQNVkzpUZqJ,0
0,7JqYAc8ACWC32Qc6JiQSSi,1p5mKMwcQVCeQNVkzpUZqJ,1
0,5P5xuOwnMwaTY38yT8laSl,1p5mKMwcQVCeQNVkzpUZqJ,2
0,0wIrCNimJJJsIn5HFYOAnW,1p5mKMwcQVCeQNVkzpUZqJ,3
0,6kLJ2eRnzga0vepfLkD4NH,1p5mKMwcQVCeQNVkzpUZqJ,4
...,...,...,...
0,6u5lHozM0oGj2l16lTF3KP,1p5mKMwcQVCeQNVkzpUZqJ,1070
0,7oAuqs6akGnPU3Tb00ZmyM,1p5mKMwcQVCeQNVkzpUZqJ,1071
0,38vxstSXciei0IuEO2MoGI,1p5mKMwcQVCeQNVkzpUZqJ,1072
0,2xOdFPQn17smnZrFCtFNSv,1p5mKMwcQVCeQNVkzpUZqJ,1073


In [25]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Power Down Playlist with Sian Eleri")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Power Down Playlist with Sian Eleri"
description = "Songs from the Power Down Playlist with Sian Eleri in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,0aTsSP2JiY53i0uZM24Zwp,3Tqe2k7TlEB5OatSMKeppu,0
0,0TLAptKgYxe7F0KewWH6X6,3Tqe2k7TlEB5OatSMKeppu,1
0,6Oryv6qqkDSi0opyTi7gvJ,3Tqe2k7TlEB5OatSMKeppu,2
0,3J0jlN9j7fpmghvALVF0Th,3Tqe2k7TlEB5OatSMKeppu,3
0,4R2kfaDFhslZEMJqAFNpdd,3Tqe2k7TlEB5OatSMKeppu,4
...,...,...,...
0,1TLHZcMQRMgW3N7hjh9Zlu,3Tqe2k7TlEB5OatSMKeppu,670
0,0tNuJpBgtE65diL6Q8Q7fI,3Tqe2k7TlEB5OatSMKeppu,671
0,3Z3wyUUPcKyw4nJkW2PqBA,3Tqe2k7TlEB5OatSMKeppu,672
0,3INuZoAMZsz0BOiEFTLmAu,3Tqe2k7TlEB5OatSMKeppu,673


In [45]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == 'BBC Introducing on Radio 1')].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = 'BBC Introducing on Radio 1'
description = "Songs from BBC Introducing on Radio 1 in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,0MjMYoSWmtVYAhv1Uj3zQO,07B5mpuLVdVpgSycsqn4Qc,0
0,01nBevWzm0Hs0kEFAkxkh4,07B5mpuLVdVpgSycsqn4Qc,1
0,2CgajbdKtE8HCvzra4t7Pb,07B5mpuLVdVpgSycsqn4Qc,2
0,4Ds6xHWHN5tU8Rxg7na56i,07B5mpuLVdVpgSycsqn4Qc,3
0,1QpVsWAQJDM5dX9bV6qtFt,07B5mpuLVdVpgSycsqn4Qc,4
...,...,...,...
0,2kC9Ccwk1ILSzCN3IBROOx,07B5mpuLVdVpgSycsqn4Qc,634
0,4f5SAiUVc4WFvuxva3JhnR,07B5mpuLVdVpgSycsqn4Qc,635
0,3BOiQtFYI4EE27CyNwm9rU,07B5mpuLVdVpgSycsqn4Qc,636
0,62UTBl5LVd6HOCFvT57Axt,07B5mpuLVdVpgSycsqn4Qc,637


In [26]:
#make_playlist(title, description)
#edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

#edit_playlist_details(user=my_id, playlist_id = playlist_id, name = name', description = 'description')
#get_playlist(playlist_id, with_tracks = False)
#get_playlist(playlist_id = playlist_id)
#delete_playlist(playlist_id = playlist_id)

In [57]:
playlist_tracks.to_pickle('../data/spotify_playlist_tracks.pkl')
playlists.to_pickle('../data/spotify_playlists.pkl')
tracks.to_pickle('../data/spotify_tracks.pkl')
albums.to_pickle('../data/spotify_albums.pkl')
artists.to_pickle('../data/spotify_artists.pkl')